# **連結google 雲端資料**

In [1]:
# # 取得google drive存取權限
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

# Tapiei_QA data

In [2]:
# with open('/content/drive/Shared drives/新手村/data/Taipei_QA_new.txt') as file:
#   data = file.readlines()  # 以 \n 為一個單位讀取
with open('./Taipei_QA_new.txt') as file:
    data = file.readlines()  # 以 \n 為一個單位讀取
  

In [3]:
for i,line in enumerate(data):
    if i < 20:
        print(line)

In [4]:
for i, line in enumerate(data):
    sp = line.strip().split(' ')
    if (i%500==0) :
        print('label',sp[0])
        print('question',sp[1]) 
        print()
#     if i == 10 :break 

label 臺北市政府文化局
question 2018臺北藝術節FAQ

label 臺北市政府社會局婦女福利及兒童托育科
question 居家式托育服務登記需要提出申請嗎？何時開始受理？去哪裡申請？

label 臺北市政府社會局人民團體科
question 申請籌組人民團體應備文件為何?

label 臺北市政府觀光傳播局
question 只要是出租套房都可以檢舉嗎？

label 臺北市政府衛生局疾病管制科
question 參加臺北市營業場所衛生優良自主管理分級認證有何好處？

label 臺北大眾捷運股份有限公司
question 於捷運系統內遺失及找尋遺失物之相關處理方式？

label 臺北市政府民政局戶籍行政科
question 戶政事務所夜間服務時間及項目為何？

label 臺北市立聯合醫院
question 想做乳癌篩檢，不知如何辨理？

label 臺北市政府勞動局職業安全衛生科
question 若發現工地未作好安全防護有危害勞工安全或導致公共危險之虞該如何處理？

label 臺北市政府兵役局
question ●應徵役男延期徵集入營事故表

label 臺北市動物保護處動物管理組
question 送交動物之家收容之流浪犬貓是否皆有確實作晶片掃描與通知主人？另未植入晶片之犬貓，飼主要如何查詢是否已遭安樂死處理？

label 臺北市立文獻館
question 古物有哪些法令規定？

label 臺北市藝文推廣處
question 如何查詢藝文推廣處城市舞台檔期？

label 臺北市交通事件裁決所案件管理課
question 我要如何辦理將交通違規案件移轉給實際駕駛人（不是我違規駕駛，卻收到紅單，舉發的時間與地點，我根本不在那裡）？需要攜帶什麼文件？

label 臺北市政府地政局地價科
question 是否所有受理申報登錄之實際資訊，皆會對外公開？公開的方式為何?會不會洩漏個人隱私？

label 臺北市動物保護處防疫檢驗組
question 家裡如果有養寵物鳥的人要怎麼預防禽流感？



In [5]:
num = 0
label = []     
for i, line in enumerate(data):
    num += 1
    sp = line.strip().split(' ')
    if sp[0] not in label:
        label.append(sp[0])
print('總數',num)
print('類別',len(label))

總數 7986
類別 149


# 資料前處理

In [6]:
x=[]
y=[]
for i, line in enumerate(data):
    sp = line.strip().split(' ')
    x.append(sp[1])     #Q
    y.append(sp[0])     #A
print(x[0],y[0],len(x),len(y))

2018臺北藝術節FAQ 臺北市政府文化局 7986 7986


input 處理

In [7]:
#使用實驗室api斷詞 https://github.com/UDICatNCHU/UdicOpenData
!pip3 install udicOpenData
from udicOpenData.dictionary import *
from udicOpenData.stopwords import *

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.520 seconds.
Prefix dict has been built succesfully.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
for i,ele in enumerate(x):
    print(ele)
    sen_list = list(rmsw(ele))
    print(sen_list)
    print()
    if i == 5 :break

2018臺北藝術節FAQ
['臺北', '藝術節', 'FAQ']

臺北市信義區Neo19大樓後方人行道後場使用作業通告(107/06/03-05)
['臺北市', '信義區', 'Neo', '大樓', '後方', '人行道', '後場', '使用', '作業', '通告']

2018台北電影節FAQ
['台北', '電影節', 'FAQ']

臺灣新文化運動紀念館位置與聯絡方式
['臺灣', '新文化運動', '紀念館', '位置', '聯絡', '方式']

臺灣新文化運動紀念館開館及參觀時間?
['臺灣', '新文化運動', '紀念館', '開館', '參觀', '時間']

2018臺北兒童藝術節FAQ
['臺北', '兒童', '藝術節', 'FAQ']



In [9]:
from gensim import models
model = models.Word2Vec.load('./model/word2vec_wiki_zh.model.bin')

In [10]:
#將input斷詞並轉成word2vec向量
x_vector = []
for i,q in enumerate(x):
    word_list = list(rmsw(q))
    tmp=[]
    for word in word_list:
        try:
            tmp.append(model[word])
        except:
            continue
    x_vector.append(tmp)

# print(len(tmp),len(tmp[0]))
# print(tmp)
# break

print(len(x_vector))

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


7986


In [11]:
#input 長度不一樣須補成相同大小
print(len(x_vector[0]),len(x_vector[1]),len(x_vector[2]),len(x_vector[3]))

2 9 1 6


In [12]:
# 找出input最大的長度
max_len=0
for v in x_vector:
    if len(v) > max_len: max_len=len(v)
print(max_len)

# 找出平均input之長度
total_len=0
for v in range(len(x_vector)):
    total_len += len(x_vector[v])
print(total_len/len(x_vector))

60
8.303531179564237


In [13]:
#將input補齊至最大長度
import numpy as np

for v in x_vector:
    while len(v) < 10:
        v.append(np.zeros(400,dtype=float))
    v = v[:10]
print(len(x_vector[0]),len(x_vector[1]),len(x_vector[2]),len(x_vector[3]))
# 只留下前10個input
for i in range(len(x_vector)):
    x_vector[i] = x_vector[i][:10]
print(len(x_vector[0]),len(x_vector[1]),len(x_vector[2]),len(x_vector[3]))


10 10 10 10
10 10 10 10


label 處理

In [14]:
#將label 做 LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

y_Encode = le.fit_transform(y) #y為答案 就是類別
print(y_Encode[0], y_Encode[100], y_Encode[200])

#反轉回文字
print(le.inverse_transform([y_Encode[0], y_Encode[100], y_Encode[200]]))

78 100 110
['臺北市政府文化局' '臺北市政府產業發展局科技產業服務中心' '臺北市政府社會局身心障礙者福利科']


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


切training data 與 testing data

In [15]:
from sklearn.model_selection import train_test_split
#Sklearn-train_test_split隨機劃分訓練集和測試集 https://blog.csdn.net/CherDW/article/details/54881167
x_train, x_test, y_train, y_test = train_test_split(x_vector, y_Encode, test_size=0.1)

print("train數",len(x_train)," test數",len(x_test))

train數 7187  test數 799


In [16]:
import torch
# new_tensor = torch.Tensor([[0,0],[0,0]])
# print(x_train[0][0])
x_train_tensor=torch.FloatTensor(x_train)
y_train_tensor=torch.LongTensor(y_train)
x_test_tensor=torch.Tensor(x_test)
y_test_tensor=torch.LongTensor(y_test)

# PyTorch code

In [17]:
from torch import nn

input_size = 400  # input的維度
hidden_dim = 400  # 隱藏層維度
num_layers = 1    # GRU迭代次數
label_num = 149   # 總Label數量

max_len = 10      # 句子最大長度
batch_size = 8    # batch_size

gru = nn.GRU(input_size, hidden_dim, num_layers=1, batch_first=True) # input_size,  隱藏層維度

gru_input = torch.randn(batch_size, max_len, input_size) # batch_size, 句子最大長度, input的維度



h0 = torch.zeros(num_layers, batch_size, hidden_dim) # (num_layers, batch, hidden_dim)

gru_output, hidden = gru(gru_input,h0)

print(gru_output.shape, hidden.shape)

senVec_output = gru_output[:, -1, :] #只使用最後的输出做為句向量

print(senVec_output.shape)

liner = nn.Linear(hidden_dim, label_num)    # 將句向量經過一層liner判斷類別
liner_output = liner(senVec_output)

print(liner_output.shape)



torch.Size([8, 10, 400]) torch.Size([1, 8, 400])
torch.Size([8, 400])
torch.Size([8, 149])


In [18]:
import torch.utils.data as Data
BATCH_SIZE = 50
train_dataset = Data.TensorDataset(x_train_tensor, y_train_tensor)
loader = Data.DataLoader(
    dataset = train_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True,
)
test_dataset = Data.TensorDataset(x_test_tensor, y_test_tensor)
t_loader = Data.DataLoader(
    dataset = test_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True,
)

# 網路架構

In [19]:
import torch
from torch import nn
import torch.nn.functional as F

class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()

        self.gru = nn.GRU(         # if use nn.RNN(), it hardly learns
            input_size=400,
            hidden_size=400,         # rnn hidden unit
            num_layers=1,           # number of rnn layer
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )

        self.out = nn.Linear(400,149)
        
    def forward(self, x):
        r_out,_= self.gru(x, None)  # None represents zero initial hidden state
        
        
        out = F.relu(self.out(r_out[:, -1, :]))
        
        
        return out


rnn = RNN()
print(rnn)

RNN(
  (gru): GRU(400, 400, batch_first=True)
  (out): Linear(in_features=400, out_features=149, bias=True)
)


# train

In [20]:
import numpy as np
def accuracy(logits, labels):
    outputs = np.argmax(logits, axis=1)
    num_correct = torch.eq(outputs, labels).sum().item()
    return num_correct

In [21]:
LR = 0.0003
optimizer = torch.optim.AdamW(rnn.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss()

for epoch in range(30):
    train_loss = 0.0
    train_correct = 0
    rnn.train()
    for step, (x, y) in enumerate(loader):
#         print(x.shape)
        output = rnn(x)
        train_correct += accuracy(torch.tensor(output),y)
    
        loss = loss_func(output, y)
        train_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#     print(train_loss/step)
        
    test_loss = 0.0
    test_correct = 0
    rnn.eval()
    for step, (x, y) in enumerate(t_loader):
        with torch.no_grad():
            output = rnn(x)
            loss = loss_func(output, y)
        test_loss += loss.item()
        test_correct += accuracy(output, y)

    print(
    'epoch:',epoch+1,
    'train_loss:',round(train_loss/len(loader), 4),
    'train_acc:',round(train_correct/len(train_dataset), 4),
    'test_loss:',round(test_loss/len(t_loader), 4),
    'test_acc:',round(test_correct/len(test_dataset), 4)    
    )
        

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':


epoch: 1 train_loss: 4.6393 train_acc: 0.0792 test_loss: 4.1857 test_acc: 0.1727
epoch: 2 train_loss: 3.79 train_acc: 0.25 test_loss: 3.5082 test_acc: 0.2991
epoch: 3 train_loss: 3.2003 train_acc: 0.3573 test_loss: 3.1214 test_acc: 0.3579
epoch: 4 train_loss: 2.8036 train_acc: 0.4387 test_loss: 2.8822 test_acc: 0.4005
epoch: 5 train_loss: 2.415 train_acc: 0.5118 test_loss: 2.6366 test_acc: 0.428
epoch: 6 train_loss: 2.0667 train_acc: 0.5852 test_loss: 2.4812 test_acc: 0.4606
epoch: 7 train_loss: 1.7796 train_acc: 0.6494 test_loss: 2.395 test_acc: 0.4944
epoch: 8 train_loss: 1.5613 train_acc: 0.705 test_loss: 2.3405 test_acc: 0.5044
epoch: 9 train_loss: 1.3835 train_acc: 0.7436 test_loss: 2.318 test_acc: 0.5156
epoch: 10 train_loss: 1.2282 train_acc: 0.7765 test_loss: 2.3098 test_acc: 0.5219
epoch: 11 train_loss: 1.0932 train_acc: 0.8072 test_loss: 2.304 test_acc: 0.5232
epoch: 12 train_loss: 0.9805 train_acc: 0.8314 test_loss: 2.3109 test_acc: 0.5232
epoch: 13 train_loss: 0.8944 train_

# 網路儲存

In [22]:
torch.save(rnn,"rnn.pkl")

/usr/local/lib/python3.5/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.5/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type GRU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.5/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Linear. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [ ]:
trained = torch.load("rnn.pkl")
while True:
    # 存下input的向量
    a=input("問題:")
    word_vec = list(rmsw(a))
    print(word_vec)
    tmp = []
    for word in word_vec:
        try:
            tmp.append(model[word])
        except:
            continue

    while len(tmp)<10:
        tmp.append(np.zeros(400,dtype=float))

    tmp = tmp[:10]

    # 丟入網路
    tmp_tensor = torch.Tensor([tmp])
    li = trained(tmp_tensor).tolist()[0]
    print(li)
    class_value = li.index(max(li))
    print("相關局處:",le.inverse_transform([class_value]))

問題:診所、藥局在臺北內湖科技園區設置之規定為何?


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


['診所', '藥局', '臺北', '內湖科技園區', '設置', '規定', '為何']
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.056695934385061264, 0.08180554211139679, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9925494194030762, 1.1749348640441895, 0.0, 2.5930373668670654, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1135234385728836, 1.1916658878326416, 0.0, 0.0, 0.48978087306022644, 2.9751360416412354, 4.674118518829346, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.040330410003662, 0.0, 0.17874856293201447, 0.0, 0.0, 0.9844323992729187, 0.0, 0.0, 0.0, 0.0, 3.659186363220215, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8727205991744995, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5850470066070557, 1.6534565687179565, 0.0, 0.39732515811920166, 2.4462618827819824, 1.655893325805664, 0.0, 0.0, 0.0, 0.805568277835846, 0.0, 0.1620902121067047, 2.7334630489349365, 2.1340491771698, 10.248215675354004, 0.36413294076919556, 0.0, 1.7337937355041504, 2.528071880340576, 0.0, 0.0,

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


['使用', '本市', '失能', '身心障礙', '日間', '照顧', '中心', '資格', '為何']
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.754073143005371, 3.6789369583129883, 1.331752896308899, 2.6624255180358887, 0.0, 0.0, 2.0237984657287598, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9091451168060303, 0.0, 0.8446336388587952, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2338891178369522, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9569203853607178, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9246938228607178, 1.379549264907837, 0.0, 2.4801244735717773, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.4730082750320435, 0.0, 1.5630829334259033, 1.352105736732483, 0.8455406427383423, 0.0, 0.0, 0.0, 2.118631362915039, 1.4148914813995361, 0.0, 0.27066826820373535, 0.0, 0.0, 0.0, 0.0, 2.2784531116485596, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.9046190977096558, 2.5042543411254883, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5060649514198303, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.7346508502960205, 0.0, 0.0, 1.088938593864441, 0.0, 0.0, 2.455970048904419, 0.0, 0.03186297044157982, 0.3559908270

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


['護理', '之家', '有無', '空', '床位', '申請', '入住']
[0.0, 0.0, 0.0, 1.6212931871414185, 0.0, 0.0, 0.0, 7.300157070159912, 1.008862853050232, 4.112353324890137, 2.752234935760498, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.5234336853027344, 0.0, 0.8529079556465149, 7.278026103973389, 0.0, 0.05331875756382942, 0.0, 0.0, 0.0, 0.0, 1.7923346757888794, 0.0, 0.3439505100250244, 0.0, 0.0, 5.774489879608154, 0.0, 0.0, 1.4977205991744995, 0.0, 3.9149439334869385, 0.0, 0.0, 0.0, 1.266196846961975, 2.99326491355896, 2.553163766860962, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.489365816116333, 2.4007229804992676, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.7981698513031006, 0.04427178204059601, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.7078373432159424, 3.1562604904174805, 0.0, 0.6351320147514343, 0.0, 5.30369234085083, 0.0, 0.0, 0.7807392477989197, 2.692464590072632, 0.3199706971645355, 2.3027446269989014, 0.49506616592407227, 0.0, 0.0, 0.0, 0.0, 1.4006065130233765, 0.0, 0.0, 0.0